Install GSSHApy development version: https://github.com/ci-water/gsshapy

In [15]:
from datetime import datetime, timedelta
import os

import geopandas as gpd
from gsshapy.modeling import GSSHAModel
import quest

Function to automate the downloading process:

In [2]:
def download_data(source_uri, bounds, collection_name):
    """
    Downloads data from source uri and adds to collection
    """
    features = quest.api.get_features(uris=source_uri,
                                      filters={'bbox': bounds},
                                      as_dataframe=True,
                                      )
    added_features = quest.api.add_features(collection_name, features)
    staged_datasets = quest.api.stage_for_download(uris=added_features)
    quest.api.download_datasets(datasets=staged_datasets)
    final_datasets = staged_datasets
    if len(staged_datasets) > 1:
        merged_datasets = quest.api.apply_filter(name='raster-merge',
                                                 datasets=staged_datasets)
        final_datasets = merged_datasets['datasets']
        # delete merged datasets
        quest.api.delete(staged_datasets)

    downloaded_datasets = quest.api.get_metadata(final_datasets)
    downloaded_file_paths = [dataset['file_path'] for dataset
                             in downloaded_datasets.values()]
    return downloaded_file_paths[0]

Parameters to change for the run:

In [13]:
elevation_service = 'svc://usgs-ned:13-arc-second'
land_use_service = 'svc://usgs-nlcd:2011'
land_use_grid_id = 'nlcd'
base_dir = '/home/rdchlads/scripts/quest/examples'
boundary_shapefile = os.path.join(base_dir, 'vicksburg_watershed', 'watershed_boundary.shp')
gssha_model_name = 'vicksburg_south'
gssha_model_directory = os.path.join(base_dir, gssha_model_name)

Get the download boundary from the shapefile:

In [4]:
boundary_gpd = gpd.read_file(boundary_shapefile)
bounds = [str(x) for x in boundary_gpd.geometry.bounds.values[0]]

The collection is where we will origanize the downloaded data.

In [5]:
try:
    quest.api.new_collection(gssha_model_name)
except ValueError as e:
    print(e)

Download the data. This may take some time.

In [7]:
elevation_file_path = download_data(elevation_service, bounds, gssha_model_name)

retrieving raster availability from https://www.sciencebase.gov/catalog/items?parentId=4f70aa9fe4b058caae3f8de5&filter=tags%3DIMG&max=1000&fields=webLinks%2Cspatial%2Ctitle&format=json&filter=spatialQuery%3D%7Bwkt%3A%22POLYGON+%28%28-180.0+90.0%2C-180.0+-90.0%2C180.0+-90.0%2C180.0+90.0%2C-180.0+90.0%29%29%22%2Crelation%3A%22intersects%22%7D
retrieving raster availability from https://www.sciencebase.gov/catalog/items?filter=tags%3DIMG&filter=spatialQuery%3D%7Bwkt%3A%22POLYGON+%28%28-180.0+90.0%2C-180.0+-90.0%2C180.0+-90.0%2C180.0+90.0%2C-180.0+90.0%29%29%22%2Crelation%3A%22intersects%22%7D&max=1000&format=json&fields=webLinks%2Cspatial%2Ctitle&parentId=4f70aa9fe4b058caae3f8de5&offset=1000
... ... .img format raster saved at /home/rdchlads/.local/share/quest/projects/default/vicksburg_south/d9aba68106c44619809f98909268c1e7.img
... ... .img format raster saved at /home/rdchlads/.local/share/quest/projects/default/vicksburg_south/dcf62f7b6ef54da588f89edb0fe9d221.img


In [8]:
land_use_file_path = download_data(land_use_service, bounds, gssha_model_name)

... ... .tif format raster saved at /home/rdchlads/.local/share/quest/projects/default/vicksburg_south/d8a67562528943b2ad79b1f87e5b06cb.tif


Use the data to generate a GSSHA model. This may take a minute.

In [16]:
# make the directory for the output
try:
    os.mkdir(gssha_model_directory)
except OSError:
    pass

# generate GSSHA model files
model = GSSHAModel(project_name=gssha_model_name,
                   project_directory=gssha_model_directory,
                   mask_shapefile=boundary_shapefile,
                   elevation_grid_path=elevation_file_path,
                   land_use_grid=land_use_file_path,
                   land_use_grid_id=land_use_grid_id,
                   out_hydrograph_write_frequency=1,
                   )

# add card for max depth
model.project_manager.setCard('FLOOD_GRID',
                              '{0}.fgd'.format(gssha_model_name),
                              add_quotes=True)
# TODO: Add depth grids to simulation
# MAP_FREQ, DEPTH

# add event for simulation
model.set_event(simulation_start=datetime.utcnow(),
                simulation_duration=timedelta(seconds=2*60),
                rain_intensity=24,
                rain_duration=timedelta(seconds=1*60),
                )
model.write()